In [7]:
# %pip install pandas-datareader fredapi yfinance apscheduler requests

In [6]:
import pandas as pd
import pandas_datareader.data as web
from fredapi import Fred
import requests
import io
from datetime import datetime

class MacroAgent:
    def __init__(self, fred_api_key=None):
        """
        初始化 MacroAgent
        :param fred_api_key: FRED API Key (必填，否則無法抓取大部分數據)
        """
        self.fred = Fred(api_key=fred_api_key) if fred_api_key else None
        
        # 定義我們要監控的指標清單 (FRED Code: 中文說明)
        self.indicators_map = {
            # --- 利率與資金成本 (Interest Rates) ---
            'FEDFUNDS': 'US_Fed_Funds_Rate',       # 聯邦基金利率 (基準利率)
            'DGS10': 'US_10Y_Treasury_Yield',      # 10年期公債殖利率 (全球資產定價之錨)
            'DGS2': 'US_2Y_Treasury_Yield',        # 2年期公債殖利率 (對利率政策最敏感)
            'T10Y2Y': 'US_10Y_Minus_2Y',           # 殖利率曲線倒掛 (經濟衰退預警)
            'WALCL': 'Fed_Balance_Sheet',          # 聯準會資產負債表規模 (印鈔/縮表指標)
            
            # --- 通膨 (Inflation) ---
            'CPIAUCSL': 'US_CPI',                  # 消費者物價指數 (影響升息)
            'PCEPI': 'US_PCE',                     # 個人消費支出 (Fed最看重的通膨指標)
            'PPIFIS': 'US_PPI',                    # 生產者物價指數 (CPI的前哨站)
            
            # --- 實體經濟 (Real Economy) ---
            'PAYEMS': 'US_NonFarm_Payrolls',       # 非農就業人口 (每月最重要數據)
            'UNRATE': 'US_Unemployment_Rate',      # 失業率
            'INDPRO': 'US_Industrial_Production',  # 工業生產指數
            'RSXFS': 'US_Retail_Sales',            # 零售銷售 (消費力道)
            
            # --- 情緒與領先指標 (Sentiment & Leading) ---
            'UMCSENT': 'US_Consumer_Sentiment',    # 密西根大學消費者信心指數
            'VIXCLS': 'VIX_Index',                 # 恐慌指數
            'M2SL': 'US_M2_Money_Supply',          # M2貨幣供給 (資金流動性)
            
            # --- 匯率與其他 (FX & Others) ---
            'DTWEXBGS': 'US_Dollar_Index_Broad',   # 美元指數 (廣義)
            'DEXTAUS': 'USD_TWD',                  # 美元兌台幣匯率 (外資進出參考)
        }

    def fetch_us_macro_data(self, start_date='2010-01-01'):
        """
        從 FRED 抓取美國總經數據
        """
        if not self.fred:
            raise ValueError("請提供 FRED API Key 才能執行此功能")
            
        print(f"[{datetime.now()}] 開始抓取美國總經數據...")
        
        # 由於不同數據更新頻率不同 (日/月/週)，我們分開抓取後再合併
        df_combined = pd.DataFrame()

        for code, name in self.indicators_map.items():
            try:
                # 抓取數據
                series = self.fred.get_series(code, observation_start=start_date)
                series.name = name
                
                # 將 Series 轉為 DataFrame 並合併
                df_temp = pd.DataFrame(series)
                
                # 這裡使用 'outer' join，保留所有時間點的數據
                if df_combined.empty:
                    df_combined = df_temp
                else:
                    df_combined = df_combined.join(df_temp, how='outer')
                    
                print(f"  -> 已獲取: {name} ({code})")
            except Exception as e:
                print(f"  -> 錯誤: 無法獲取 {code}. Reason: {e}")

        # 數據清洗：補值 (Forward Fill) 
        # 總經數據通常是公告後直到下次公告前都有效，所以用前值填充
        df_combined = df_combined.ffill()
        
        # 移除太舊的空值
        df_combined = df_combined.dropna(how='all')
        
        print(f"[{datetime.now()}] 美國數據抓取完成。")
        return df_combined

    def fetch_taiwan_business_indicators(self):
        """
        抓取台灣景氣對策信號 (國發會)
        來源：台灣政府資料開放平台 (data.gov.tw)
        """
        print(f"[{datetime.now()}] 開始抓取台灣景氣對策信號...")
        
        # 這是國發會 "景氣指標及對策信號" 的 CSV API URL (需定期維護確認 URL 有效性)
        # 這裡使用示範的固定 URL，實際生產環境建議用 requests 搜尋 data.gov.tw API
        url = "https://ws.ndc.gov.tw/Download.ashx?u=LzAwMS9hZG1pbmlzdHJhdG9yLzEwL3JlbGZpbGUvNTU2Ni82ODExLzAwNjUxNTVfMS5jc3Y%3d&n=MjAyNDEx5pmv5rCj5omR562W5L%2b66Jmf5Yia5p6Q57e66KGoLmNzdg%3d%3d"
        # 上面的 URL 經常變動，為了穩定性，我們通常建議使用 finlab 或爬蟲去 ndc.gov.tw 抓最新連結
        # 這裡為了演示代碼結構，我們模擬一個簡單的數據生成，或者如果 URL 失效請手動替換
        
        try:
            # 假設我們找到了一個穩定的 CSV 來源
            # 這裡演示如何處理這種 CSV 格式
            # 實際專案中，台灣數據建議使用 'FinLab' 套件會更穩定
            pass 
        except Exception as e:
            print(f"台灣數據抓取失敗: {e}")
            return None

    def generate_report(self, df):
        """簡單分析當前狀態"""
        latest = df.iloc[-1]
        print("\n========== 總經情勢速報 ==========")
        print(f"日期: {latest.name.date()}")
        print(f"1. 10年債殖利率: {latest['US_10Y_Treasury_Yield']}% (若 > 4% 對科技股不利)")
        print(f"2. 殖利率倒掛 (10Y-2Y): {latest['US_10Y_Minus_2Y']} (若 < 0 代表衰退風險)")
        print(f"3. 恐慌指數 (VIX): {latest['VIX_Index']} (若 > 20 代表市場動盪)")
        print(f"4. 台幣匯率: {latest['USD_TWD']} (若貶值趨勢，外資易賣超)")
        print("==================================")

# --- 執行範例 ---
if __name__ == "__main__":
    # 請替換成你申請的 FRED API KEY
    MY_API_KEY = "3e2d2e27e5126fac34a02e9edaa80c2e" 
    
    if MY_API_KEY == "YOUR_FRED_API_KEY_HERE":
        print("請先去 https://fred.stlouisfed.org/docs/api/api_key.html 申請 API Key")
    else:
        agent = MacroAgent(fred_api_key=MY_API_KEY)
        
        # 1. 抓取數據
        macro_data = agent.fetch_us_macro_data(start_date='2020-01-01')
        
        # 2. 顯示前幾筆
        print(macro_data.tail())
        
        # 3. 產生報告
        agent.generate_report(macro_data)
        
        # 4. 存檔 (這就是你的 Database)
        macro_data.to_csv("f://macro_data.csv")
        print("數據已保存至 macro_data.csv")

[2025-12-26 15:41:27.226998] 開始抓取美國總經數據...
  -> 已獲取: US_Fed_Funds_Rate (FEDFUNDS)
  -> 已獲取: US_10Y_Treasury_Yield (DGS10)
  -> 已獲取: US_2Y_Treasury_Yield (DGS2)
  -> 已獲取: US_10Y_Minus_2Y (T10Y2Y)
  -> 已獲取: Fed_Balance_Sheet (WALCL)
  -> 已獲取: US_CPI (CPIAUCSL)
  -> 已獲取: US_PCE (PCEPI)
  -> 已獲取: US_PPI (PPIFIS)
  -> 已獲取: US_NonFarm_Payrolls (PAYEMS)
  -> 已獲取: US_Unemployment_Rate (UNRATE)
  -> 已獲取: US_Industrial_Production (INDPRO)
  -> 已獲取: US_Retail_Sales (RSXFS)
  -> 已獲取: US_Consumer_Sentiment (UMCSENT)
  -> 已獲取: VIX_Index (VIXCLS)
  -> 已獲取: US_M2_Money_Supply (M2SL)
  -> 已獲取: US_Dollar_Index_Broad (DTWEXBGS)
  -> 已獲取: USD_TWD (DEXTAUS)
[2025-12-26 15:41:38.588950] 美國數據抓取完成。
            US_Fed_Funds_Rate  US_10Y_Treasury_Yield  US_2Y_Treasury_Yield  \
2025-12-18               3.88                   4.12                  3.46   
2025-12-19               3.88                   4.16                  3.48   
2025-12-22               3.88                   4.17                  3.44   
2025